## Sun Feb 10 4:44 am
* Initial pass at an LSTM based on hydraseq
* It has context, which builds a hirarchical tree of convos at high level
* Still need to define state a little better with active/predicted
* Still need to use context, add to it so we can handle she/it etc from one sentence to the next

In [1]:
! pwd
import sys
sys.path.append("/Users/niarfe/tmprepos/hydra_inc/hydraseq")
import hydraseq
from hydraseq import Hydraseq
from hydraseq.columns import run_convolutions

/Users/niarfe/tmprepos/hydra_inc/hydraseq/notebooks


In [2]:
# sentence = "the quick brown fox jumped over the lazy dog"
sentence = "spring leaves spring"
source_file = "seasons.txt"

In [3]:
class LSTM:
    def __init__(self, datasource=None):
        self.hdq = hydraseq.Hydraseq('one')
        self.sentence = ""
        self.convos0 = []
        self.convos1 = []
        self.convos2 = []
        if datasource: self.consume_data(datasource)
        self.sdrs = [
            self.sentence,
            self.convos0,
            self.convos1,
            self.convos2
        ]
        self.context = []
        
    
    def consume_data(self, fpath):
        with open(fpath, 'r') as source:
            [self.hdq.insert(line.strip()) for line in source]
            
    def process_sentence(self, sentence):
        self.sdrs = []
        self.sdrs.append(sentence.split())

        self.convos0 = run_convolutions(sentence.split(), self.hdq, "_")
        self.sdrs.append(self.convos0)
        encoded = list(map(lambda x: x[2], self.convos0))

        self.convos1 = run_convolutions(list(encoded), self.hdq, "*")
        self.sdrs.append(self.convos1)
        encoded = list(map(lambda x: x[2], self.convos1))
            
        self.convos2 = run_convolutions(encoded, self.hdq, "#")
        self.sdrs.append(self.convos2)
        
        for convo in self.convos2:
            tree = {}
            self.treeify(tree, 3, convo)
            self.context.append(tree)
        return self

    def treeify(self, tree, level, convo):
        if level == 0:
            #print("returning with ", convo)
            tree['word'] = convo
            return convo
        #print("calling self.sdrs[{}][{}:{}]".format(level-1, convo[0], convo[1]))
        next_convos = self.sdrs[level-1][convo[0]:convo[1]]
        current_dict = {}
        tree[convo[2][0]] = current_dict
        for conv in next_convos:
            self.treeify(current_dict, level-1, conv)
        
    def __str__(self):
        for sdr in self.sdrs:
            print(sdr)
            print()
        print(self.context)
        return "OK"


In [4]:
lstm = LSTM(datasource=source_file)

lstm.process_sentence(sentence)

print(lstm)

['spring', 'leaves', 'spring']

[[0, 1, ['_ADJ', '_NOU', '_VER']], [1, 2, ['_NOU', '_VER']], [2, 3, ['_ADJ', '_NOU', '_VER']]]

[[0, 2, ['*NP*', '*VP*']], [1, 3, ['*VP*']]]

[[0, 2, ['#SENT']]]

[{'#SENT': {'*NP*': {'_ADJ': {'word': 'spring'}, '_NOU': {'word': 'leaves'}}, '*VP*': {'_NOU': {'word': 'leaves'}, '_ADJ': {'word': 'spring'}}}}]
OK


In [5]:
def marmalade(d_tree):
    sep = 0
    for parent_key, child_dict in d_tree.items():
        #print('d ', parent_key, child_dict)
        if 'word' in child_dict.keys():
            print(parent_key, "-->", child_dict['word'])
            continue
        else:
            for grand_name, grand_dict in child_dict.items():
                print(parent_key, "-->", grand_name)
            marmalade(child_dict)
        sep += 1
        
for d_tree in lstm.context:
    marmalade(d_tree)
    print("------------")

#SENT --> *NP*
#SENT --> *VP*
*NP* --> _ADJ
*NP* --> _NOU
_ADJ --> spring
_NOU --> leaves
*VP* --> _NOU
*VP* --> _ADJ
_NOU --> leaves
_ADJ --> spring
------------
